In [1]:
import argparse

import json
import os
import pickle as pkl
import sys
import time
import warnings

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import uproot
from coffea import nanoevents, processor
from coffea.nanoevents import BaseSchema, NanoAODSchema, NanoEventsFactory

sys.path.append("../")

import json
import os
import pathlib
import pickle as pkl
import shutil
import warnings
from collections import defaultdict
from typing import List, Optional

import awkward as ak
import numpy as np
import pandas as pd
import pyarrow as pa
from coffea import processor
from coffea.analysis_tools import PackedSelection, Weights
from coffea.nanoevents.methods import candidate, vector

warnings.filterwarnings("ignore", message="Found duplicate branch ")
warnings.filterwarnings("ignore", category=DeprecationWarning)
np.seterr(invalid="ignore")


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
! ls ../rootfiles

hww1.root  hww2.root


In [3]:
# load a root file into coffea-friendly NanoAOD structure
import uproot
f = uproot.open(f"../rootfiles/hww1.root")
num = f['Events'].num_entries   ### checks number of events per file 
print(f'number of events per file is {num}')

events = nanoevents.NanoEventsFactory.from_root(f, "Events").events()

number of events per file is 98400


In [4]:
def build_p4(cand):
    return ak.zip(
        {
            "pt": cand.pt,
            "eta": cand.eta,
            "phi": cand.phi,
            "mass": cand.mass,
            "charge": cand.charge,
        },
        with_name="PtEtaPhiMCandidate",
        behavior=candidate.behavior,
    )

In [5]:
### make selections
nevents = len(events)

# define muon objects
loose_muons = (
    (((events.Muon.pt > 30) & (events.Muon.pfRelIso04_all < 0.25)) |
     (events.Muon.pt > 55))
    & (np.abs(events.Muon.eta) < 2.4)
    & (events.Muon.looseId)
)
n_loose_muons = ak.sum(loose_muons, axis=1)

good_muons = (
    (events.Muon.pt > 28)
    & (np.abs(events.Muon.eta) < 2.4)
    & (np.abs(events.Muon.dz) < 0.1)
    & (np.abs(events.Muon.dxy) < 0.05)
    & (events.Muon.sip3d <= 4.0)
    & events.Muon.mediumId
)
n_good_muons = ak.sum(good_muons, axis=1)

# define electron objects
loose_electrons = (
    (((events.Electron.pt > 38) & (events.Electron.pfRelIso03_all < 0.25)) |
     (events.Electron.pt > 120))
    & ((np.abs(events.Electron.eta) < 1.44) | (np.abs(events.Electron.eta) > 1.57))
    & (events.Electron.cutBased >= events.Electron.LOOSE)
)
n_loose_electrons = ak.sum(loose_electrons, axis=1)

good_electrons = (
    (events.Electron.pt > 38)
    & ((np.abs(events.Electron.eta) < 1.44) | (np.abs(events.Electron.eta) > 1.57))
    & (np.abs(events.Electron.dz) < 0.1)
    & (np.abs(events.Electron.dxy) < 0.05)
    & (events.Electron.sip3d <= 4.0)
    & (events.Electron.mvaFall17V2noIso_WP90)
)
n_good_electrons = ak.sum(good_electrons, axis=1)

# leading lepton
goodleptons = ak.concatenate([events.Muon[good_muons], events.Electron[good_electrons]], axis=1)
goodleptons = goodleptons[ak.argsort(goodleptons.pt, ascending=False)]
candidatelep = ak.firsts(goodleptons)

# candidate leptons
candidatelep_p4 = build_p4(candidatelep)

# MET
met = events.MET
mt_lep_met = np.sqrt(
    2. * candidatelep_p4.pt * met.pt * (ak.ones_like(met.pt) - np.cos(candidatelep_p4.delta_phi(met)))
)

# JETS
goodjets = events.Jet[
    (events.Jet.pt > 30)
    & (abs(events.Jet.eta) < 2.5)
    & events.Jet.isTight
    & (events.Jet.puId > 0)
]
ht = ak.sum(goodjets.pt, axis=1)

# FATJETS
fatjets = events.FatJet

good_fatjets = (
    (fatjets.pt > 200)
    & (abs(fatjets.eta) < 2.5)
    & fatjets.isTight
)
n_fatjets = ak.sum(good_fatjets, axis=1)

good_fatjets = fatjets[good_fatjets]
good_fatjets = good_fatjets[ak.argsort(good_fatjets.pt, ascending=False)]

# for lep channel: first clean jets and leptons by removing overlap, then pick candidate_fj closest to the lepton
lep_in_fj_overlap_bool = good_fatjets.delta_r(candidatelep_p4) > 0.1
good_fatjets = good_fatjets[lep_in_fj_overlap_bool]
fj_idx_lep = ak.argmin(good_fatjets.delta_r(candidatelep_p4), axis=1, keepdims=True)
candidatefj = ak.firsts(good_fatjets[fj_idx_lep])

In [6]:
candidatefj.pt

<Array [None, None, None, ... None, None, None] type='98400 * ?float32[parameter...'>

In [7]:
from typing import Dict
from coffea.nanoevents.methods.base import NanoEventsArray

def get_pfcands_features(
    tagger_vars: dict,
    preselected_events: NanoEventsArray,
    fj_idx_lep,
    fatjet_label: str = "FatJetAK15",
    pfcands_label: str = "FatJetPFCands",
    normalize: bool = True,
) -> Dict[str, np.ndarray]:
    """
    Extracts the pf_candidate features specified in the ``tagger_vars`` dict from the
    ``preselected_events`` and returns them as a dict of numpy arrays
    """

    feature_dict = {}

    jet = ak.firsts(preselected_events[fatjet_label][fj_idx_lep])

    msk = preselected_events[pfcands_label].jetIdx == ak.firsts(fj_idx_lep)
    jet_ak_pfcands = preselected_events[pfcands_label][msk]
    jet_pfcands = preselected_events.PFCands[jet_ak_pfcands.pFCandsIdx]

    # sort them by pt
    pfcand_sort = ak.argsort(jet_pfcands.pt, ascending=False)
    jet_pfcands = jet_pfcands[pfcand_sort]

    # negative eta jets have -1 sign, positive eta jets have +1
    eta_sign = ak.ones_like(jet_pfcands.eta)
    eta_sign = eta_sign * (ak.values_astype(jet.eta > 0, int) * 2 - 1)
    feature_dict["pfcand_etarel"] = eta_sign * (jet_pfcands.eta - jet.eta)
    feature_dict["pfcand_phirel"] = jet.delta_phi(jet_pfcands)
    feature_dict["pfcand_abseta"] = np.abs(jet_pfcands.eta)

    feature_dict["pfcand_pt_log_nopuppi"] = np.log(jet_pfcands.pt)
    feature_dict["pfcand_e_log_nopuppi"] = np.log(jet_pfcands.energy)

    pdgIds = jet_pfcands.pdgId
    feature_dict["pfcand_isEl"] = np.abs(pdgIds) == 11
    feature_dict["pfcand_isMu"] = np.abs(pdgIds) == 13
    feature_dict["pfcand_isChargedHad"] = np.abs(pdgIds) == 211
    feature_dict["pfcand_isGamma"] = np.abs(pdgIds) == 22
    feature_dict["pfcand_isNeutralHad"] = np.abs(pdgIds) == 130

    feature_dict["pfcand_charge"] = jet_pfcands.charge
    feature_dict["pfcand_VTX_ass"] = jet_pfcands.pvAssocQuality
    feature_dict["pfcand_lostInnerHits"] = jet_pfcands.lostInnerHits
    feature_dict["pfcand_quality"] = jet_pfcands.trkQuality

    feature_dict["pfcand_normchi2"] = np.floor(jet_pfcands.trkChi2)

    if "Cdz" in jet_ak_pfcands.fields:
        feature_dict["pfcand_dz"] = jet_ak_pfcands["Cdz"][pfcand_sort]
        feature_dict["pfcand_dxy"] = jet_ak_pfcands["Cdxy"][pfcand_sort]
        feature_dict["pfcand_dzsig"] = jet_ak_pfcands["Cdzsig"][pfcand_sort]
        feature_dict["pfcand_dxysig"] = jet_ak_pfcands["Cdxysig"][pfcand_sort]
    else:
        # this is for old PFNano (<= v2.3)
        feature_dict["pfcand_dz"] = jet_pfcands.dz
        feature_dict["pfcand_dxy"] = jet_pfcands.d0
        feature_dict["pfcand_dzsig"] = jet_pfcands.dz / jet_pfcands.dzErr
        feature_dict["pfcand_dxysig"] = jet_pfcands.d0 / jet_pfcands.d0Err

    feature_dict["pfcand_px"] = jet_pfcands.px
    feature_dict["pfcand_py"] = jet_pfcands.py
    feature_dict["pfcand_pz"] = jet_pfcands.pz
    feature_dict["pfcand_energy"] = jet_pfcands.E
    # feature_dict["pfcand_energy"] = jet_pfcands.energy

    # btag vars
    for var in tagger_vars["pf_features"]["var_names"]:
        if "btag" in var:
            feature_dict[var] = jet_ak_pfcands[var[len("pfcand_") :]][pfcand_sort]

    # pfcand mask
    feature_dict["pfcand_mask"] = (
        ~(
            ma.masked_invalid(
                ak.pad_none(
                    feature_dict["pfcand_abseta"],
                    tagger_vars["pf_features"]["var_length"],
                    axis=1,
                    clip=True,
                ).to_numpy()
            ).mask
        )
    ).astype(np.float32)

    # if no padding is needed, mask will = 1.0
    if isinstance(feature_dict["pfcand_mask"], np.float32):
        feature_dict["pfcand_mask"] = np.ones(
            (
                len(feature_dict["pfcand_abseta"]),
                tagger_vars["pf_features"]["var_length"],
            )
        ).astype(np.float32)

    repl_values_dict = {
        "pfcand_normchi2": [-1, 999],
        "pfcand_dz": [-1, 0],
        "pfcand_dzsig": [1, 0],
        "pfcand_dxy": [-1, 0],
        "pfcand_dxysig": [1, 0],
    }

    # convert to numpy arrays and normalize features
    if "pf_vectors" in tagger_vars.keys():
        variables = set(tagger_vars["pf_features"]["var_names"] + tagger_vars["pf_vectors"]["var_names"])
    else:
        variables = tagger_vars["pf_features"]["var_names"]

    for var in variables:
        a = (
            ak.pad_none(
                feature_dict[var],
                tagger_vars["pf_features"]["var_length"],
                axis=1,
                clip=True,
            )
            .to_numpy()
            .filled(fill_value=0)
        ).astype(np.float32)
        a = np.nan_to_num(a)

        # replace values to match PKU's
        if var in repl_values_dict:
            vals = repl_values_dict[var]
            a[a == vals[0]] = vals[1]

        if normalize:
            if var in tagger_vars["pf_features"]["var_names"]:
                info = tagger_vars["pf_features"]["var_infos"][var]
            else:
                info = tagger_vars["pf_vectors"]["var_infos"][var]

            a = (a - info["median"]) * info["norm_factor"]
            a = np.clip(a, info.get("lower_bound", -5), info.get("upper_bound", 5))

        feature_dict[var] = a

    return feature_dict

# Gen Matching

In [8]:
from typing import Union
from coffea.nanoevents.methods.nanoaod import FatJetArray, GenParticleArray

def get_pid_mask(
    genparts: GenParticleArray,
    pdgids: Union[int, list],
    ax: int = 2,
    byall: bool = True,
) -> ak.Array:
    """
    Get selection mask for gen particles matching any of the pdgIds in ``pdgids``.
    If ``byall``, checks all particles along axis ``ax`` match.
    """
    gen_pdgids = abs(genparts.pdgId)

    if type(pdgids) == list:
        mask = gen_pdgids == pdgids[0]
        for pdgid in pdgids[1:]:
            mask = mask | (gen_pdgids == pdgid)
    else:
        mask = gen_pdgids == pdgids

    return ak.all(mask, axis=ax) if byall else mask

d_PDGID = 1
c_PDGID = 4
b_PDGID = 5
g_PDGID = 21
TOP_PDGID = 6

ELE_PDGID = 11
vELE_PDGID = 12
MU_PDGID = 13
vMU_PDGID = 14
TAU_PDGID = 15
vTAU_PDGID = 16

Z_PDGID = 23
W_PDGID = 24
HIGGS_PDGID = 25

PI_PDGID = 211
PO_PDGID = 221
PP_PDGID = 111

GEN_FLAGS = ["fromHardProcess", "isLastCopy"]

FILL_NONE_VALUE = -99999

JET_DR = 0.8

def to_label(array: ak.Array) -> ak.Array:
    return ak.values_astype(array, np.int32)

In [9]:
genparts = events.GenPart
fatjet = candidatefj

In [10]:
dau_pdgid=W_PDGID

"""Gen matching for Higgs samples"""
higgs = genparts[get_pid_mask(genparts, HIGGS_PDGID, byall=False) * genparts.hasFlags(GEN_FLAGS)]

# only select events that match a specific decay
signal_mask = ak.firsts(ak.all(abs(higgs.children.pdgId) == dau_pdgid, axis=2))

matched_higgs = higgs[ak.argmin(fatjet.delta_r(higgs), axis=1, keepdims=True)][:, 0]
matched_higgs_children = matched_higgs.children

genVars = {"fj_genH_pt": ak.fill_none(matched_higgs.pt, FILL_NONE_VALUE)}

if dau_pdgid == W_PDGID:
    children_mask = get_pid_mask(matched_higgs_children, [W_PDGID], byall=False)
    matched_higgs_children = matched_higgs_children[children_mask]

    # order by mass, select lower mass child as V* and higher as V
    children_mass = matched_higgs_children.mass
    v_star = ak.firsts(matched_higgs_children[ak.argmin(children_mass, axis=1, keepdims=True)])
    v = ak.firsts(matched_higgs_children[ak.argmax(children_mass, axis=1, keepdims=True)])

    is_hww_matched = ak.any(children_mask, axis=1)

    genVVars = {
        "fj_genV_dR": fatjet.delta_r(v),
        "fj_genVstar": fatjet.delta_r(v_star),
        "genV_genVstar_dR": v.delta_r(v_star),
    }

    # VV daughters
    daughters = ak.flatten(matched_higgs_children.distinctChildren, axis=2)
    daughters = daughters[daughters.hasFlags(GEN_FLAGS)]
    daughters_pdgId = abs(daughters.pdgId)

    # exclude neutrinos from nprongs count
    daughters_nov = daughters[
        ((daughters_pdgId != vELE_PDGID) & (daughters_pdgId != vMU_PDGID) & (daughters_pdgId != vTAU_PDGID))
    ]
    nprongs = ak.sum(fatjet.delta_r(daughters_nov) < JET_DR, axis=1)

    # get tau decays
    taudaughters = daughters[(daughters_pdgId == TAU_PDGID)].children
    taudaughters = taudaughters[taudaughters.hasFlags(["isLastCopy"])]
    taudaughters_pdgId = abs(taudaughters.pdgId)

    taudecay = (
        # pions/kaons (hadronic tau) * 1
        (
            ak.sum(
                (taudaughters_pdgId == ELE_PDGID) | (taudaughters_pdgId == MU_PDGID),
                axis=1,
            )
            == 0
        )
        * 1
        # 1 electron * 3
        + (ak.sum(taudaughters_pdgId == ELE_PDGID, axis=1) == 1) * 3
        # 1 muon * 5
        + (ak.sum(taudaughters_pdgId == MU_PDGID, axis=1) == 1) * 5
    )
    # flatten taudecay - so painful
    taudecay = ak.sum(taudecay, axis=-1)

    # lepton daughters
    lepdaughters = daughters[
        ((daughters_pdgId == ELE_PDGID) | (daughters_pdgId == MU_PDGID) | (daughters_pdgId == TAU_PDGID))
    ]
    lepinprongs = ak.sum(fatjet.delta_r(lepdaughters) < JET_DR, axis=1)  # should be 0 or 1

    lepton_parent = ak.firsts(lepdaughters[fatjet.delta_r(lepdaughters) < JET_DR].distinctParent)
    lepton_parent_mass = lepton_parent.mass

    iswlepton = lepton_parent_mass == v.mass
    iswstarlepton = lepton_parent_mass == v_star.mass    
    
    decay = (
        # 2 quarks * 1
        (ak.sum(daughters_pdgId <= b_PDGID, axis=1) == 2) * 1
        # 1 electron * 3
        + (ak.sum(daughters_pdgId == ELE_PDGID, axis=1) == 1) * 3
        # 1 muon * 5
        + (ak.sum(daughters_pdgId == MU_PDGID, axis=1) == 1) * 5
        # 1 tau * 7
        + (ak.sum(daughters_pdgId == TAU_PDGID, axis=1) == 1) * 7
        # 4 quarks * 11
        + (ak.sum(daughters_pdgId <= b_PDGID, axis=1) == 4) * 11
    )

    # number of c quarks in V decay inside jet
    cquarks = daughters_nov[abs(daughters_nov.pdgId) == c_PDGID]
    ncquarks = ak.sum(fatjet.delta_r(cquarks) < JET_DR, axis=1)

    genHVVVars = {
        "fj_nprongs": nprongs,
        "fj_ncquarks": ncquarks,
        "fj_lepinprongs": lepinprongs,
        "fj_H_VV_4q": to_label(decay == 11),
        "fj_H_VV_elenuqq": to_label(decay == 4),
        "fj_H_VV_munuqq": to_label(decay == 6),
        "fj_H_VV_leptauelvqq": to_label((decay == 8) & (taudecay == 3)),
        "fj_H_VV_leptaumuvqq": to_label((decay == 8) & (taudecay == 5)),
        "fj_H_VV_hadtauvqq": to_label((decay == 8) & (taudecay == 1)),
        "fj_H_VV_isVlepton": iswlepton,
        "fj_H_VV_isVstarlepton": iswstarlepton,
        "fj_H_VV_isMatched": is_hww_matched,
    }

    genVars = {**genVars, **genVVars, **genHVVVars}

elif dau_pdgid == TAU_PDGID:
    children_mask = get_pid_mask(matched_higgs_children, [TAU_PDGID], byall=False)
    daughters = matched_higgs_children[children_mask]

    is_htt_matched = ak.any(children_mask, axis=1)

    taudaughters = daughters[(abs(daughters.pdgId) == TAU_PDGID)].children
    taudaughters = taudaughters[taudaughters.hasFlags(["isLastCopy"])]
    taudaughters_pdgId = abs(taudaughters.pdgId)

    taudaughters = taudaughters[
        ((taudaughters_pdgId != vELE_PDGID) & (taudaughters_pdgId != vMU_PDGID) & (taudaughters_pdgId != vTAU_PDGID))
    ]
    taudaughters_pdgId = abs(taudaughters.pdgId)

    flat_taudaughters_pdgId = ak.flatten(taudaughters_pdgId, axis=2)

    extra_taus = ak.any(taudaughters_pdgId == TAU_PDGID, axis=2)
    children_pdgId = abs(taudaughters[extra_taus].children.pdgId)

    taudecay = (
        # pions/kaons (full hadronic tau) * 1
        (
            (
                ak.sum(
                    (flat_taudaughters_pdgId == PI_PDGID)
                    | (flat_taudaughters_pdgId == PO_PDGID)
                    | (flat_taudaughters_pdgId == PP_PDGID),
                    axis=1,
                )
                > 0
            )
        )
        * 1
        # 1 electron * 3
        + (ak.sum(flat_taudaughters_pdgId == ELE_PDGID, axis=1) == 1) * 3
        # 1 muon * 5
        + (ak.sum(flat_taudaughters_pdgId == MU_PDGID, axis=1) == 1) * 5
        # two leptons
        + (
            (ak.sum(flat_taudaughters_pdgId == ELE_PDGID, axis=1) == 2)
            | (ak.sum(flat_taudaughters_pdgId == MU_PDGID, axis=1) == 2)
        )
        * 7
    )

    extradecay = (
        (
            (
                ak.sum(
                    ak.sum(
                        (children_pdgId == PI_PDGID) | (children_pdgId == PO_PDGID) | (children_pdgId == PP_PDGID),
                        axis=-1,
                    ),
                    axis=1,
                )
                > 0
            )
        )
        * 1
        + (ak.sum(ak.sum(children_pdgId == ELE_PDGID, axis=-1), axis=1) == 1) * 3
        + (ak.sum(ak.sum(children_pdgId == MU_PDGID, axis=-1), axis=1) == 1) * 5
        + (
            (ak.sum(ak.sum(children_pdgId == MU_PDGID, axis=-1), axis=1) == 2)
            | (ak.sum(ak.sum(children_pdgId == ELE_PDGID, axis=-1), axis=1) == 2)
        )
        * 7
    )
    extradecay = ak.sum(extradecay, axis=-1)

    elehad = ((taudecay == 4) & (extradecay == 0)) | ((extradecay == 4) & (taudecay == 0))
    muhad = ((taudecay == 6) & (extradecay == 0)) | ((extradecay == 6) & (taudecay == 0))
    leplep = ((taudecay == 7) | (taudecay == 8)) | ((extradecay == 7) | (extradecay == 8))
    hadhad = ~elehad & ~muhad & ~leplep

    # to painfully debug
    # np.set_printoptions(threshold=np.inf)
    # print(ak.argsort((is_htt_matched)).to_numpy())
    # print(ak.flatten(taudecay).to_numpy())
    # idx= ak.argsort((is_htt_matched)).to_numpy()
    # idx = [74,2023,2037,2887,3121,3435,3838,4599,4702,4906,5266,5703,6063,6498,6799,7642,8820,8828,8999,9005,9455,9564,
    # 11178,11597,11736,12207,12325,12504,12697,12780,13151,13690]
    # for i in idx:
    #     print(i,flat_taudaughters_pdgId[i],extra_taus[i],children_pdgId[i])
    #     print(elehad[i],muhad[i],leplep[i],hadhad[i])
    #     print(taudecay[i],extradecay[i])

    genHTTVars = {
        "fj_H_tt_hadhad": to_label(hadhad),
        "fj_H_tt_elehad": to_label(elehad),
        "fj_H_tt_muhad": to_label(muhad),
        "fj_H_tt_leplep": to_label(leplep),
        "fj_H_tt_isMatched": is_htt_matched,
    }

    genVars = {**genVars, **genHTTVars}

In [11]:
genVars

{'fj_genH_pt': <Array [-1e+05, -1e+05, ... -1e+05, -1e+05] type='98400 * float64'>,
 'fj_genV_dR': <Array [None, None, None, ... None, None, None] type='98400 * ?float32'>,
 'fj_genVstar': <Array [None, None, None, ... None, None, None] type='98400 * ?float32'>,
 'genV_genVstar_dR': <Array [None, None, None, ... None, None, None] type='98400 * ?float32'>,
 'fj_nprongs': <Array [None, None, None, ... None, None, None] type='98400 * ?int64'>,
 'fj_ncquarks': <Array [None, None, None, ... None, None, None] type='98400 * ?int64'>,
 'fj_lepinprongs': <Array [None, None, None, ... None, None, None] type='98400 * ?int64'>,
 'fj_H_VV_4q': <Array [None, None, None, ... None, None, None] type='98400 * ?int32'>,
 'fj_H_VV_elenuqq': <Array [None, None, None, ... None, None, None] type='98400 * ?int32'>,
 'fj_H_VV_munuqq': <Array [None, None, None, ... None, None, None] type='98400 * ?int32'>,
 'fj_H_VV_leptauelvqq': <Array [None, None, None, ... None, None, None] type='98400 * ?int32'>,
 'fj_H_VV_